<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/lt_whisper_medium_test_(edit).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-91vc2vyr
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-91vc2vyr
  Resolved https://github.com/huggingface/transformers to commit 238d2e3c44366aba9dc5c770c95475765a6725cb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [4]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [5]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "lt", split="test")
#column_name = "audio, normalized_text"

In [6]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 42
})

In [7]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [22]:
model = whisper.load_model("medium")

In [23]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [24]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [25]:
text

,0
0,niekur pasaulyje konkrečiai kalbant apie lenkų...
1,prieš dvidešimt trys metus sausio trylika ąją ...
2,atsižvelgiant į šiandieninę situaciją tiesiog ...
3,noriu pabrėžti reaguodama į kai kurių kolegų p...
4,taip pat ir toliau privalėsime stebėti ir užti...
5,tuo tarpu europos sąjunga nesustoja klydusi sa...
6,tačiau nepaisant skausmingų mūsų istorijų taut...
7,viena iš priežasčių pieno supirkėjų ir prekybi...
8,tik mūsų bendros pastangos teisingi prioriteta...
9,taip aš sakiau kad kalbu ecr frakcijos vardu t...


In [26]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [27]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,female,niekur pasaulyje konkrečiai kalbant apie lenkų...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000274658203125, -0.001129150390625, 0.000...",16000
1,female,prieš dvidešimt trys metus sausio trylika ąją ...,/root/.cache/huggingface/datasets/downloads/ex...,"[3.0517578125e-05, -0.003265380859375, -0.0012...",16000
2,male,atsižvelgiant į šiandieninę situaciją tiesiog ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0001220703125, -0.000213623046875, 0.0, -0...",16000
3,female,noriu pabrėžti reaguodama į kai kurių kolegų p...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001556396484375, 0.001068115234375, -0.0011...",16000
4,male,taip pat ir toliau privalėsime stebėti ir užti...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.006591796875, -0.00653076171875, 0.0028076...",16000
5,male,tuo tarpu europos sąjunga nesustoja klydusi sa...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.14666748046875, -0.121917724609375, -0.118...",16000
6,male,tačiau nepaisant skausmingų mūsų istorijų taut...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001312255859375, -0.00042724609375, -0.000...",16000
7,male,viena iš priežasčių pieno supirkėjų ir prekybi...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00067138671875, -0.000762939453125, -0.000...",16000
8,male,tik mūsų bendros pastangos teisingi prioriteta...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000640869140625, -0.0009765625, -0.0004272...",16000
9,male,taip aš sakiau kad kalbu ecr frakcijos vardu t...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002716064453125, 0.008087158203125, 0.01113...",16000


In [28]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
2,male,atsižvelgiant į šiandieninę situaciją tiesiog ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0001220703125, -0.000213623046875, 0.0, -0...",16000
4,male,taip pat ir toliau privalėsime stebėti ir užti...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.006591796875, -0.00653076171875, 0.0028076...",16000
5,male,tuo tarpu europos sąjunga nesustoja klydusi sa...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.14666748046875, -0.121917724609375, -0.118...",16000
6,male,tačiau nepaisant skausmingų mūsų istorijų taut...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001312255859375, -0.00042724609375, -0.000...",16000
7,male,viena iš priežasčių pieno supirkėjų ir prekybi...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00067138671875, -0.000762939453125, -0.000...",16000
8,male,tik mūsų bendros pastangos teisingi prioriteta...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000640869140625, -0.0009765625, -0.0004272...",16000
9,male,taip aš sakiau kad kalbu ecr frakcijos vardu t...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002716064453125, 0.008087158203125, 0.01113...",16000
10,male,baltarusija privalo nedelsiant sustabdyti stat...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000152587890625, -0.00018310546875, 0.0004...",16000
11,male,kalbant apie federalistines ambicijas ar tiksl...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.040435791015625, 0.040679931640625, 0.03750...",16000
12,male,todėl artėjant armėnų genocido šimtmečio pamin...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00103759765625, -0.001312255859375, -0.001...",16000


In [29]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
0,female,niekur pasaulyje konkrečiai kalbant apie lenkų...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000274658203125, -0.001129150390625, 0.000...",16000
1,female,prieš dvidešimt trys metus sausio trylika ąją ...,/root/.cache/huggingface/datasets/downloads/ex...,"[3.0517578125e-05, -0.003265380859375, -0.0012...",16000
3,female,noriu pabrėžti reaguodama į kai kurių kolegų p...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001556396484375, 0.001068115234375, -0.0011...",16000
19,female,vilniaus susitikimas buvo labai reikšmingas st...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002838134765625, 0.009124755859375, 0.00411...",16000
20,female,labai smagu girdėti tiek daug sveikinimų lietu...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.030364990234375, 0.028045654296875, 0.02862...",16000
23,female,todėl prašau nebeklaidinti kolegų ir nevelti j...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000213623046875, -0.001495361328125, -0.00...",16000
25,female,todėl labai noriu padėkoti visai lietuvos koma...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0074462890625, -0.003875732421875, 0.00210...",16000
30,female,dabar turime atsižvelgti į didžiųjų veikėjų ža...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000152587890625, 0.0008544921875, 0.0010681...",16000
31,female,ir turbūt stebuklas kad šiandien po dvidešimt ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.006744384765625, -0.00274658203125, -0.001...",16000
38,female,manau kad pasiekėme pirmininkavimo uždavinius ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00714111328125, -0.00177001953125, -0.00933...",16000


In [30]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="lt", fp16=False)
    return result["text"]

In [31]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [32]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [33]:

total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.5652173913043478, Precision: 0.47619047619047616, Recall: 0.43478260869565216, F1: 0.4545454545454545
Processed 2 files, WER: 0.6111111111111112, Precision: 0.375, Recall: 0.35294117647058826, F1: 0.3636363636363636
Processed 3 files, WER: 1.0, Precision: 0.14285714285714285, Recall: 0.1111111111111111, F1: 0.125
Processed 4 files, WER: 0.4090909090909091, Precision: 0.6190476190476191, Recall: 0.5909090909090909, F1: 0.6046511627906977
Processed 5 files, WER: 0.7272727272727273, Precision: 0.3333333333333333, Recall: 0.38095238095238093, F1: 0.35555555555555557
Processed 6 files, WER: 0.5384615384615384, Precision: 0.5555555555555556, Recall: 0.4166666666666667, F1: 0.4761904761904762
Processed 7 files, WER: 0.5, Precision: 0.5428571428571428, Recall: 0.5277777777777778, F1: 0.5352112676056338
Processed 8 files, WER: 0.5, Precision: 0.6111111111111112, Recall: 0.6111111111111112, F1: 0.6111111111111112
Processed 9 files, WER: 0.30303030303030304, Precision: 0

In [34]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)


    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.391304347826087, Precision: 0.7142857142857143, Recall: 0.7142857142857143, F1: 0.7142857142857143
Processed 2 files, WER: 0.75, Precision: 0.29411764705882354, Recall: 0.2631578947368421, F1: 0.27777777777777773
Processed 3 files, WER: 0.35294117647058826, Precision: 0.6875, Recall: 0.6470588235294118, F1: 0.6666666666666667
Processed 4 files, WER: 0.18181818181818182, Precision: 0.8181818181818182, Recall: 0.8181818181818182, F1: 0.8181818181818182
Processed 5 files, WER: 0.5238095238095238, Precision: 0.5555555555555556, Recall: 0.5555555555555556, F1: 0.5555555555555556
Processed 6 files, WER: 0.2, Precision: 0.8, Recall: 0.8, F1: 0.8000000000000002
Processed 7 files, WER: 0.4666666666666667, Precision: 0.5172413793103449, Recall: 0.5769230769230769, F1: 0.5454545454545454
Processed 8 files, WER: 0.4166666666666667, Precision: 0.5833333333333334, Recall: 0.6363636363636364, F1: 0.6086956521739131
Processed 9 files, WER: 0.75, Precision: 0.2857142857142857,